In [ ]:
%matplotlib notebook
import numpy as np
import geojson
import rasterio
import firedrake
import icepack, icepack.plot

Load the outline and create a mesh.

In [ ]:
outline_filename = icepack.datasets.fetch_pine_island_outline()
with open(outline_filename, 'r') as outline_file:
    outline = geojson.load(outline_file)
    
geometry = icepack.meshing.collection_to_geo(outline)
with open('pine-island.geo', 'w') as geo_file:
    geo_file.write(geometry.get_code())

In [ ]:
!gmsh -2 -format msh2 -v 0 -o pine-island.msh pine-island.geo

In [ ]:
mesh = firedrake.Mesh('pine-island.msh')

In [ ]:
fig, axes = icepack.plot.subplots()
icepack.plot.triplot(mesh, axes=axes)
axes.legend()

Load the bed and thickness maps and apply a smoothing kernel.

In [ ]:
bedmachine_filename = icepack.datasets.fetch_bedmachine_antarctica()
thickness = rasterio.open('netcdf:' + bedmachine_filename + ':thickness', 'r')
bed = rasterio.open('netcdf:' + bedmachine_filename + ':bed', 'r')

In [ ]:
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)
h_obs = icepack.interpolate(thickness, Q)
b_obs = icepack.interpolate(bed, Q)

In [ ]:
from firedrake import inner, grad, dx

def smooth(q_obs, α):
    q = q_obs.copy(deepcopy=True)
    r = q - q_obs
    J = 0.5 * (inner(r, r) + α**2 * inner(grad(q), grad(q))) * dx
    F = firedrake.derivative(J, q)
    firedrake.solve(F == 0, q)
    return q

α = firedrake.Constant(2e3)
h = smooth(h_obs, α)
b = smooth(b_obs, α)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(h, 40, axes=axes)
fig.colorbar(contours)

In [ ]:
s = icepack.compute_surface(h=h, b=b)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(s, 40, axes=axes)
fig.colorbar(contours)

Calculate the height of the ice surface above flotation.

In [ ]:
from icepack.constants import ice_density as ρ_I, water_density as ρ_W
h_f = firedrake.interpolate(s - (1 - ρ_I / ρ_W) * h, Q)

Show all the regions where the height above flotation is less than 200m.

In [ ]:
fig, axes = icepack.plot.subplots()
levels = np.linspace(0., 200., 21)
contours = icepack.plot.tricontourf(h_f, levels=levels, extend='min', axes=axes)
fig.colorbar(contours)

Load the ice velocity.

In [ ]:
velocity_filename = icepack.datasets.fetch_measures_antarctica()
vx = rasterio.open('netcdf:' + velocity_filename + ':VX', 'r')
vy = rasterio.open('netcdf:' + velocity_filename + ':VY', 'r')
stdx = rasterio.open('netcdf:' + velocity_filename + ':STDX', 'r')
stdy = rasterio.open('netcdf:' + velocity_filename + ':STDY', 'r')

In [ ]:
V = firedrake.VectorFunctionSpace(mesh, family='CG', degree=2)
u_obs = icepack.interpolate((vx, vy), V)
σx = icepack.interpolate(stdx, Q)
σy = icepack.interpolate(stdy, Q)

Calculate the driving stress and smooth it a lot.
This is to get the initial guess for the friction coefficient.

In [ ]:
from icepack.constants import gravity as g
from firedrake import grad
τ = firedrake.Function(V)
r = τ + ρ_I * g * h * grad(s)
α = firedrake.Constant(10e3)
J = 0.5 * (inner(r, r) + α**2 * inner(grad(τ), grad(τ))) * dx
F = firedrake.derivative(J, τ)
firedrake.solve(F == 0, τ)

In [ ]:
fig, axes = icepack.plot.subplots()
levels = np.linspace(0., 0.2, 41)
contours = icepack.plot.tricontourf(τ, levels=levels, extend='both', axes=axes)
fig.colorbar(contours)

Make an initial guess for the sliding coefficient -- the bed takes up half of the driving stress.
(You can tune the fraction but it's just a starting place.)
Note that we're using the Weertman sliding law

$$\tau_b = -C|u|^{1/m - 1}u$$

where $C$ is the friction coefficient.

In [ ]:
from icepack.constants import weertman_sliding_law as m
speed = firedrake.sqrt(inner(u_obs, u_obs))
stress = firedrake.sqrt(inner(τ, τ))
fraction = firedrake.Constant(0.5)
C = fraction * stress / speed**(1/m)

Now make the parameter $\beta$, which is what we'll actually infer.
Since the friction coefficient really can be 0, we'll use the parameterization

$$C = \beta^2.$$

Additionally, multiply by a ramping factor to make the friction coefficient 0 on the ice shelf.

In [ ]:
from firedrake import max_value, min_value
h_f_max = firedrake.Constant(10.)
ramp = max_value(0, min_value(1, h_f / h_f_max))
β = firedrake.interpolate(ramp * firedrake.sqrt(C), Q)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(β, 40, axes=axes)
fig.colorbar(contours)

In [ ]:
h_f_max = firedrake.Constant(10.)
def friction(u, h, s, β):
    h_f = s - (1 - ρ_I / ρ_W) * h
    ramp = max_value(0, min_value(1, h_f / h_f_max))
    C = ramp * β**2
    return icepack.models.friction.bed_friction(u=u, C=C)

In [ ]:
model = icepack.models.IceStream(friction=friction)

In [ ]:
T = firedrake.Constant(260.)
A = icepack.rate_factor(T)

In [ ]:
opts = {'dirichlet_ids': [2, 3, 4]}
u = model.diagnostic_solve(u0=u_obs, h=h, s=s, A=A, β=β, **opts)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(u, 40, axes=axes)
fig.colorbar(contours)